## 4.4. 自定义层
深度学习的一个魅力在于神经网络中各式各样的层，例如全连接层和后面章节中将要介绍的卷积层、池化层与循环层。虽然`tf.keras`提供了大量常用的层，但有时候我们依然希望自定义层。本节将介绍如何自定义一个层，从而可以被重复调用。

### 4.4.1. 不含模型参数的自定义层

我们先介绍如何定义一个不含模型参数的自定义层。事实上，这和“模型构造”一节中介绍的使用`Model`类构造模型类似。下面的`CenteredLayer`类通过继承`keras.Model`类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了`call`函数里。这个层里不含模型参数。

In [1]:
import tensorflow as tf
from tensorflow import keras

class CenteredLayer(keras.Model):
    def __init__(self):
        super().__init__()
    
    def call(self,x) -> tf.Variable:
        return x-tf.reduce_mean(x)

我们可以实例化这个层，然后做前向计算。

In [2]:
layer = CenteredLayer()
layer(tf.Variable([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

我们也可以用它来构造更复杂的模型。

In [3]:
net = keras.Sequential()
net.add(keras.layers.Flatten())
net.add(keras.layers.Dense(128))
net.add(CenteredLayer())

下面打印自定义层各个输出的均值。因为均值是浮点数，所以它的值是一个很接近0的数。

In [4]:
y = net(tf.random.uniform(shape=(4,8),dtype=tf.float32))
tf.reduce_mean(y).numpy()

-4.656613e-09

### 4.4.2. 含模型参数的自定义层
我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。现在我们尝试实现一个含权重参数和偏差参数的全连接层。它使用`ReLU`函数作为激活函数。其中`in_units`和`units`分别代表输入个数和输出个数。[参考文档](https://keras-cn.readthedocs.io/en/latest/legacy/layers/writting_layer/)

In [5]:
class myDense(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):     # 这里 input_shape 是第一次运行call()时参数inputs的形状
        self.w = self.add_weight(name='w',
            shape=[input_shape[-1], self.units], initializer=tf.random_normal_initializer())
        self.b = self.add_weight(name='b',
            shape=[self.units], initializer=tf.zeros_initializer())

    def call(self, inputs):
        y_pred = tf.matmul(inputs, self.w) + self.b
        return y_pred

In [6]:
X = tf.random.uniform((2,20))
dense = myDense(units=3)
dense(X)
dense.get_weights()

[array([[ 9.75771155e-03,  4.00495343e-02,  3.85261490e-03],
        [-5.37358550e-03,  4.32355367e-02, -1.09703252e-02],
        [-1.90077405e-02,  2.06239652e-02, -4.99283932e-02],
        [-1.27977803e-01,  7.67421275e-02, -8.76005813e-02],
        [-8.44651908e-02,  1.20183244e-01, -3.00898533e-02],
        [ 6.69125766e-02,  3.57323252e-02,  3.26059270e-03],
        [-3.58027779e-02,  1.68202706e-02, -2.15812977e-02],
        [-2.94501130e-02,  4.59850766e-02, -4.18049563e-03],
        [-1.58748589e-03,  1.04002552e-02,  6.03588996e-03],
        [ 1.87720601e-02,  7.41220638e-02, -6.88911602e-02],
        [-1.81389730e-02,  7.58408976e-04, -1.73321739e-03],
        [ 8.69654957e-03, -1.25253145e-02, -1.68807413e-02],
        [-1.08764306e-01, -5.57432771e-02, -2.64271796e-02],
        [ 9.92755443e-02, -2.33597867e-02,  2.31827665e-02],
        [ 4.43162285e-02,  5.79406507e-02, -6.39376640e-02],
        [ 5.56709245e-02, -3.91095467e-02,  3.08088865e-02],
        [ 7.27549708e-03

我们也可以使用自定义层构造模型。

In [7]:
net = tf.keras.models.Sequential()
net.add(myDense(8))
net.add(myDense(1))

net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.0243845 ],
       [-0.01555487]], dtype=float32)>